In [2]:
import requests
from bs4 import BeautifulSoup
page = requests.get("https://www.nhtsa.gov/node/97996/251")
page

<Response [200]>

In [3]:
soup = BeautifulSoup(page.content, 'html.parser')
#print(soup.prettify())

In [4]:
# Find all hyperlink in the website
hpl= soup.find_all('a',href=True)

In [5]:
#Find all the link to each year
dl={} # variable that contains link to each year
for ele in hpl:
    e_text=ele.get_text()
    try:
        year=int(e_text)
        if (year>2013 and year <2019):
            print(year)
            print(ele['href'])
            dl[year]=ele['href']
    except: continue
    

2009
https://www.nhtsa.gov/node/97996/117196
2010
https://www.nhtsa.gov/node/97996/117201
2011
https://www.nhtsa.gov/node/97996/117206
2012
https://www.nhtsa.gov/node/97996/117211
2013
https://www.nhtsa.gov/node/97996/117216
2014
https://www.nhtsa.gov/node/97996/117221
2015
https://www.nhtsa.gov/node/97996/117226
2016
https://www.nhtsa.gov/node/97996/117231
2017
https://www.nhtsa.gov/node/97996/117236
2018
https://www.nhtsa.gov/node/97996/176766


## Download zip file of each year

In [7]:
import wget
for year in dl.keys():
    print(year)
    # find and select the national link
    page1 = requests.get(dl[year])
    soup1 = BeautifulSoup(page1.content, 'html.parser')
    hpl1= soup1.find_all('a',href=True)
    sublink=''
    for ele in hpl1:
        if ele.get_text()=='National':
            sublink=ele['href']
            break
    
    #find and select the csv file
    page1_1 = requests.get(sublink)
    soup1_1 = BeautifulSoup(page1_1.content, 'html.parser')
    hpl1_1= soup1_1.find_all('a',href=True)
    
    dll=''
    for ele in hpl1_1:
        if ele.get_text().endswith('SAS.zip'):
            dll=ele['href']
            break
    
    # Download file to the Data sets folder
    wget.download(dll,out='./Datasets')
    print('')

2009
100% [..........................................................................] 9707266 / 9707266
2010
100% [........................................................................] 13810722 / 13810722
2011
100% [........................................................................] 14099035 / 14099035
2012
100% [........................................................................] 15029317 / 15029317
2013
100% [........................................................................] 25191966 / 25191966
2014
100% [........................................................................] 25401135 / 25401135
2015
100% [........................................................................] 28731973 / 28731973
2016
100% [........................................................................] 21821873 / 21821873
2017
100% [........................................................................] 22270170 / 22270170
2018
100% [............................................

# Unzip files

In [8]:
from zipfile import ZipFile
import os

In [10]:
filelist=os.listdir('./Datasets')
dict_file=dict(zip(dl.keys(),filelist))
dict_file

{2009: 'FARS2009NationalSAS.zip',
 2010: 'FARS2010NationalSAS.zip',
 2011: 'FARS2011NationalSAS.zip',
 2012: 'FARS2012NationalSAS.zip',
 2013: 'FARS2013NationalSAS.zip',
 2014: 'FARS2014NationalSAS.zip',
 2015: 'FARS2015NationalSAS.zip',
 2016: 'FARS2016NationalSAS.zip',
 2017: 'FARS2017NationalSAS.zip',
 2018: 'FARS2018NationalSAS.zip'}

In [11]:
for year,fname in dict_file.items():
    path='./Datasets/'+fname
    zip_ref=ZipFile(path,'r')
    zip_ref.extractall('./Datasets/'+str(year))
    zip_ref.close()
    os. remove(path) # remove zip file

# Mapping and then convert to csv file

In [12]:
import pandas as pd
import pyreadstat as prs

In [31]:
os.getcwd()
files=os.listdir('.\Datasets\\2014')
print(files)

['accident.sas7bdat', 'acc_aux.sas7bdat', 'cevent.sas7bdat', 'damage.sas7bdat', 'distract.sas7bdat', 'drimpair.sas7bdat', 'factor.sas7bdat', 'Format14.sas', 'formats.sas7bcat', 'maneuver.sas7bdat', 'miacc.sas7bdat', 'midrvacc.sas7bdat', 'miper.sas7bdat', 'nmcrash.sas7bdat', 'nmimpair.sas7bdat', 'nmprior.sas7bdat', 'parkwork.sas7bdat', 'pbtype.sas7bdat', 'person.sas7bdat', 'per_aux.sas7bdat', 'safetyeq.sas7bdat', 'vehicle.sas7bdat', 'veh_aux.sas7bdat', 'vevent.sas7bdat', 'violatn.sas7bdat', 'vision.sas7bdat', 'vsoe.sas7bdat']


In [32]:
files.remove('Format10.sas')
files.remove('formats.sas7bcat')
print(files)

ValueError: list.remove(x): x not in list

In [27]:
def union_data(fname):
    df_final=pd.DataFrame()
    for year in range(2014,2019):
        path='.\Datasets\\'+str(year)
        name=''
        files=os.listdir(path)
        for file in files:
            if file.find(fname)!=-1:
                name = file
                break
        name
        path_f = path + '\\'+name
        path_c = path + '\\formats.sas7bcat'
        print(path_f,path_c)
        df= prs.read_sas7bdat(path_f, catalog_file=path_c, formats_as_category=True)[0]
        #print(df.head())
        #pd.read_csv(path,encoding = "ISO-8859-1", low_memory=False)
        df['YEAR']=year
        print(df.shape)
        df_final=pd.concat([df_final,df])
        del df
    return df_final
union_data(files[0])

.\Datasets\2018\accident.sas7bdat .\Datasets\2018\formats.sas7bcat
(33654, 52)
.\Datasets\2017\accident.sas7bdat .\Datasets\2017\formats.sas7bcat
(34560, 52)
.\Datasets\2016\accident.sas7bdat .\Datasets\2016\formats.sas7bcat
(34748, 52)
.\Datasets\2015\accident.sas7bdat .\Datasets\2015\formats.sas7bcat
(32538, 52)
.\Datasets\2014\accident.sas7bdat .\Datasets\2014\formats.sas7bcat
(30056, 50)


,STATE,ST_CASE,VE_TOTAL,VE_FORMS,PVH_INVL,PEDS,PERNOTMVIT,PERMVIT,PERSONS,COUNTY,...,ARR_HOUR,ARR_MIN,HOSP_HR,HOSP_MN,CF1,CF2,CF3,FATALS,DRUNK_DR,ROAD_FNC
0,Alabama,10001.0,2.0,1.0,1.0,0.0,0.0,1.0,1.0,121.0,...,6:00am-6:59am,15,Not Applicable (Not Transported),Not Applicable (Not Transported),None,None,None,1.0,0.0,NaN
1,Alabama,10002.0,1.0,1.0,0.0,0.0,0.0,2.0,2.0,127.0,...,0:00am-0:59am,59,Not Applicable (Not Transported),Not Applicable (Not Transported),Police Pursuit Involved,None,None,2.0,0.0,NaN
2,Alabama,10003.0,2.0,2.0,0.0,0.0,0.0,2.0,2.0,21.0,...,11:00pm-11:59pm,10,Unknown,Unknown EMS Hospital Arrival Time,None,None,None,1.0,0.0,NaN
3,Alabama,10004.0,1.0,1.0,0.0,0.0,0.0,2.0,2.0,3.0,...,1:00pm-1:59pm,14,Not Applicable (Not Transported),Not Applicable (Not Transported),None,None,None,1.0,0.0,NaN
4,Alabama,10005.0,2.0,2.0,0.0,0.0,0.0,2.0,2.0,73.0,...,7:00am-7:59am,28,Not Applicable (Not Transported),Not Applicable (Not Transported),None,None,None,1.0,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30051,Wyoming,560127.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,3.0,...,10:00pm-10:59pm,44,10:00pm-10:59pm,56,None,None,None,1.0,1.0,Rural-Principal Arterial-Other
30052,Wyoming,560128.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,15.0,...,7:00am-7:59am,47,Not Applicable (Not Transported),Not Applicable (Not Transported),None,None,None,1.0,0.0,Rural-Local Road or Street
30053,Wyoming,560129.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,37.0,...,Unknown EMS Scene Arrival Hour,Unknown EMS Scene Arrival Minutes,Not Applicable (Not Transported),Not Applicable (Not Transported),None,None,None,1.0,1.0,Rural-Principal Arterial-Interstate
30054,Wyoming,560130.0,1.0,1.0,0.0,0.0,0.0,3.0,3.0,31.0,...,Unknown EMS Scene Arrival Hour,Unknown EMS Scene Arrival Minutes,Unknown,Unknown EMS Hospital Arrival Time,None,None,None,1.0,0.0,Rural-Principal Arterial-Interstate
